# Search Engine

This lab is about starting from scratch, and hence having to structure the code yourself. A specification for the program you are to implement is given below - do pay attention as misunderstandings may cost you marks! Being precise is a necessary skill for a programmer. To give an executive summary, you are to code a search engine for recipes. A data set has been provided. The search engine is to be pretty basic, returning all recipes that contain all of the provided keywords. However, the user can choose from a number of orderings depending on their food preferences, which you need to support.

## Marking and submission

These lab exercises are marked, and contribute to your final grade. This lab exercise has 20 marks to earn, equivalent to 12% of your final grade.

Please submit your completed workbook to the auto marker before 2021-11-14 20:00 GMT. The workbook you submit must be an .ipynb file, which is saved into the directory you're running Jupyter; alternatively you can download it from the menu above using `File -> Download As -> Notebook (.ipynb)`. Remember to save your work regularly (`Save and checkpoint` in the `File` menu, the icon of a floppy disk, or `Ctrl-S`). It is wise to verify it runs to completion with _Restart & Run All_ before submission.

You must comply with the universities plagiarism guidelines: http://www.bath.ac.uk/library/help/infoguides/plagiarism.html

## Specification

The system must provide a function ``search``, with the following specification:
```
def search(query, ordering = 'normal', count = 10):
  ...
```

It `print`s out the results of the search, subject to the following rules:
1. It selects from the set of all recipes that contain __all__ of the words in the query (the positions/order of the words in the recipe are to be ignored).
2. It orders them based on the provided ordering (a string, meaning defined below).
3. It `print`s the top `count` matches only, preserving the order from best to worst. Must `print` just their title, one per line. Must be less than `count` if the search is specific enough that less than `count` recipes match.

As an aside, do not worry about memory usage. If duplicating some data can make your code faster/neater then feel free.



### Data set

A file, `recipes.json` is provided, containing 17K recipes. It can be parsed into a Python data structure using the [`json`](https://docs.python.org/3/library/json.html) module. It is a list, where each recipe is a dictionary containing various keys:
* `title` : Name of recipe; you can assume these are unique
* `categories` : A list of tags assigned to the recipe
* `ingredients` : What is in it, as a list
* `directions` : List of steps to make the recipe
* `rating` : A rating, out of 5, of how good it is
* `calories` : How many calories it has
* `protein` : How much protein is in it
* `fat` : How much fat is in it

Note that the data set was obtained via web scrapping and hence is noisy - every key except for `title` is missing from at least one recipe. Your code will need to cope with this.

You will probably want to explore the data before starting, so you have an idea of what your code has to deal with.

Data set came from https://www.kaggle.com/hugodarwood/epirecipes/version/2 though note it has been cleaned it up, by deleting duplicates and removing the really dodgy entries.



### Search

The search should check the following parts of the recipe (see data set description below):
* `title`
* `categories`
* `ingredients`
* `directions`

For instance, given the query "banana cheese" you would expect "Banana Layer Cake with Cream Cheese Frosting" in the results. Note that case is to be ignored ("banana" matches "Banana") and the words __do not__ have to be next to one another, in the same order as the search query or even in the same part of the recipe ("cheese" could appear in the title and "banana" in the ingredients). However, all words in the search query __must__ appear somewhere.



### Tokenisation

This is the term for breaking a sentence into each individual word (token). Traditionally done using regular expressions, and Python does have the `re` module, but there is no need to do that here (`re` can be quite fiddly). For matching words your tokenisation must follow the following steps:
1. Convert all punctuation and digits into spaces. For punctuation use the set in [`string.punctuation`](https://docs.python.org/3/library/string.html#string.punctuation), for digits [`string.digits`](https://docs.python.org/3/library/string.html#string.digits). You may find [`translate()`](https://docs.python.org/3/library/stdtypes.html#str.translate) interesting!
2. [`split()`](https://docs.python.org/3/library/stdtypes.html#str.split) to extract individual tokens.
3. Ignore any token that is less than $3$ characters long.
4. Make tokens lowercase.

When matching words for search (above) or ordering (below) it's only a match if you match an entire token. There are many scenarios where this simple approach will fail, but it's good enough for this exercise. The auto marker will be checking the above is followed! When doing a search the code should ignore terms in the search string that fail the above requirements.



### Ordering

There are three ordering modes to select from, each indicated by passing a string to the `search` function:
* `normal` - Based simply on the number of times the search terms appear in the recipe. A score is calculated and the order is highest to lowest. The score sums the following terms (repeated words are counted multiple times, i.e. "cheese cheese cheese" is $3$ matches to "cheese"):
    * $8 \times$ Number of times a query word appears in the title
    * $4 \times$ Number of times a query word appears in the categories
    * $2 \times$ Number of times a query word appears in the ingredients
    * $1 \times$ Number of times a query word appears in the directions
    * The `rating` of the recipe (if not available assume $0$)

* `simple` - Tries to minimise the complexity of the recipe, for someone who is in a rush. Orders to minimise the number of ingredients multiplied by the numbers of steps in the directions.

* `healthy` - Order from lowest to highest by this cost function:
$$\frac{|\texttt{calories} - 510n|}{510} + 2\frac{|\texttt{protein} - 18n|}{18} + 4\frac{|\texttt{fat} - 150n|}{150}$$
Where $n \in \mathbb{N}^+$ is selected to minimise the cost ($n$ is a positive integer and $n=0$ is not allowed). This can be understood in terms of the numbers $510$, $18$ and $150$ being a third of the recommended daily intake (three meals per day) for an average person, and $n$ being the number of whole meals the person gets out of cooking/making the recipe. So this tries to select recipes that neatly divide into a set of meals that are the right amount to consume for a healthy, balanced diet. Try not to overthink the optimisation of $n$, as it's really quite simple to do!

To clarify the use of the ordering string, to get something healthy that contains cheese you might call `search('cheese', 'healthy')`. In the case of a recipe that is missing a key in its dictionary the rules are different for each search mode:
* `normal` - Consider a missing entry in the recipe (e.g. no `ingredients` are provided) to simply mean that entry can't match any search words (because it has none!), but the item is still eligible for inclusion in the results, assuming it can match the search with a different entry.
* `simple` - If a recipe is missing either `ingredients` or `directions` it is dropped from such a search result. Because the data is messy if either of these lists is of length $1$ it should be assumed that the list extraction has failed and the recipe is to also be dropped from the search results.
* `healthy` - If any of `calories`, `protein` or `fat` is missing the recipe should be dropped from the result.



### Extra

You may find the [_inverted index_](https://en.wikipedia.org/wiki/Inverted_index) interesting. It's a data structure used by search engines. For each word a user may search for this contains a list of all documents (recipes) that contain the word. This may take a little effort to understand, but the resulting code will be both faster and neater.

## Advice

* Don't just start coding: Make a plan and work out what you intend to do.
* Think about structure, as messy code leads to mistakes.
* Plan your data structures. Don't be afraid to use sheets of paper if that works for you!
* Don't duplicate code, put it in a function/method instead.

* Divide and conquer. Break the system into parts that can implemented with minimal dependency on the rest. Functions or OOP are both suitable for doing this.
* Test early. Verify the individual parts work before trying to combine them. Factor this into the order you implement the parts of the system - don't implement something you are going to struggle to test before implementing, and verifying, other parts.
* Do not try and do a 'big bang', where you get it all working at once. Instead, get it working with features missing, then add those features in, one at a time.

* Keep things as simple as possible. Avoid long functions/methods.
* Include comments, as a form of planning and for your own sanity!
* Regularly reset the kernel and rerun the entire workbook. It is very easy to break something but not notice, because the correct version remains in memory.

## Marks
* __20 marks__: For many different _unit tests_ that will be run on `search`. They will cover all of the details in the above specification!
    * One test checks that it's faster than $0.1$ seconds on average (on the computer which runs the auto marker, which is quite fast) to do a search, so try not to be too inefficient (it ignores any time your notebook spends building data structures to be used by `search`). Note that the validation implementation comes in at $0.001$ seconds per search (after $5.5$ seconds of preparation), so this is very achievable!
    * You may want to look into Python's `set()` object, as it is useful for _one_ of the possible ways to implement this.
    * There will be sorting. The [Sorting how to](https://docs.python.org/3/howto/sorting.html) may help.
    * The auto marker does give some feedback, and you can run it as many times as you want. Don't be afraid to test an incomplete or semi-broken version of your code if stuck or unsure. It may help!
    * The validation implementation is 104 lines of code split over 5 cells (including white space for clarity and comments). Coded by someone who probably has much more experience than you, so you shouldn't aim to match this, but it's a good clue: If you find yourself at 500 lines you may want to stop and think some more! (line count does not include testing code, which is about the same amount again)

In [1]:
import json
file=open('recipes.json','r')
recipes=json.load(file)
file.close()

In [2]:
# Translante string to lower case, tokenisation, drop length < 3
import string
def Ts(query):
    s=string.digits+string.punctuation
    query=query.lower().translate(str.maketrans(s,len(s)*' ')).split()
    [query.remove(_) for _ in query[:] if len(_)<3]
    return query

In [3]:
import numpy as np
from math import ceil, floor
from numpy import inf, array
def counttoken(Q,sublist):
    tsum=0
    for q in Q:
        in_list=sublist.count(q)
        tsum+=in_list
    return tsum
    
def Score(Q,r):
    TT=counttoken(Q,r['title'])                   
    if 'categories' in r:
        CG=counttoken(Q,r['categories'])         # use .get()
    else:
        CG=0
    if 'ingredients' in r:
        IG=counttoken(Q,r['ingredients'])
    else:
        IG=0
    if 'directions' in r:
        DR=counttoken(Q,r['directions'])
    else:
        DR=0
    if 'rating' in r:
        RT=r['rating']
    else:
        RT=0
    score=np.dot([8,4,2,1,1],[TT,CG,IG,DR,RT])
    return score

def Cost(c,p,f):
    n1=c/510;n2=p/18;n3=f/150                 # they do not likely to equal
    nmin=min(n1,n2,n3)
    nmax=max(n1,n2,n3)
    # If optimal n is larger than nmax, the cost function is minimised at n=nmax. 
    # If optimal n is smaller than nmin, the cost function is minimised at n=nmin.
    # Hence try only nmin<=n<=nmax
    nlow=max(floor(nmin),1)
    nhigh=ceil(nmax)+1
    n=range(nlow,nhigh+1)                    # +1 to prevent nlow=nhigh=1
    n0=np.zeros(len(n))
    cost=abs((n0+c/510)-n)+2*abs((n0+p/18)-n)+4*abs((n0+f/150)-n)
    costmin=min(cost)
    return costmin

In [4]:
# tokenised recipes and appending extra features - tRecipes
import copy
tRecipes=copy.deepcopy(recipes)
for r,r0 in zip(tRecipes[:],recipes[:]):
    r['title']=Ts(r['title'])
    if 'categories' in r:
        r['categories']=Ts(','.join(r['categories'])) # list, repeats included         
    if 'ingredients' in r:
        r['ingredients']=Ts(','.join(r['ingredients']))
    if 'directions' in r:
        r['directions']=Ts(','.join(r['directions']))
    if 'calories' in r and 'protein' in r and 'fat' in r:              # be careful not use: ''and''and'' in r
        r['cost']=Cost(r['calories'],r['protein'],r['fat'])
        r0['cost']=r['cost']
    if 'ingredients' in r and 'directions' in r and len(r0['directions'])!=1:
        r['complexity']=len(r0['ingredients'])*len(r0['directions'])
        r0['complexity']=r['complexity']

In [5]:
# transform tokenised recipes into a list of strings
#tRecipes=[{'title':['deqca','fcfa'],'2':['dfcq']},{'3':['cqdc'],'4':4}]
K=['rating','calories','protein','fat','cost','complexity']
stRecipes=[]
for i,r in enumerate(tRecipes):
    Keys=list(r.keys())
    Values=list(r.values())
    for k,j in zip(Keys[:],Values[:]):
        if k in K:
            Values.remove(j)
    Values=sum(Values,[])
    stRecipes.append(Values)   # a list of list of strings of recipes

<ipython-input-5-793490946503>:10: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  Values.remove(j)


In [6]:
from collections import defaultdict
# find indices to recipes containing query tokens
dd=defaultdict(list)
for i,r in enumerate(stRecipes):
    for j in r:                      # have complete list of tokens of a recipe
        dd[j].append(i)
for k in dd:
    dd[k]=set(dd[k])

# x=[dd['large'],dd['cook']]
# set.intersection(*x)

In [7]:
# Sublist of recipes containing query tokens : Recipes, and query tokens
def R(query):
    Q=set(Ts(query))                         # set of tokens, repeats excluded
    I=[]
    for q in Q:
        if q in dd:
            I.append(dd[q])
    if I==[]:
        return Q,[],[]
    else:
        Index=list(set.intersection(*I))
        '''
        print(Index)
        for i in I[:]:
            if i==[]:
                I.remove(i)
        J=[]
        for i in I:
            J.append(list(i))
        Index=sum(J,[])
        '''
        Recipes=array(tRecipes)[Index].tolist()
        Recipes0=array(recipes)[Index].tolist()
        return Q,Recipes,Recipes0         # sublist of tokenised recipes

# x=R('low high Dijon rimmed baking zucchini')
# x

In [8]:
# Ordering: enter query, return full sublist (no count)
def N(query):
    Q,Recipes,Recipes0=R(query)
    Rscore=[]
    Titles=[]
    for r,r0 in zip(Recipes,Recipes0):
        rscore=Score(Q,r)
        Rscore.append(rscore)
        Titles.append(r0['title'])
    Titles=dict(zip(Rscore,Titles))                                # scores attached to recipes
    Titles=sorted(Titles.items(),key=lambda _:_[0],reverse=True)  # sort high to low
    Titles=list(array(Titles).reshape((len(Titles),2))[:,1])             
    return Titles

In [9]:
def S(query):
    Q,Recipes,Recipes0=R(query)
    for r,r0 in zip(Recipes,Recipes0[:]):
        if 'ingredients' not in r or 'directions' not in r or len(r0['directions'])==1:
            Recipes0.remove(r0)
    Recipes0.sort(key=lambda _:_['complexity'])               # sort low to high
    Titles=[r['title'] for r in Recipes0]
    return Titles

In [10]:
def H(query):
    Q,Recipes,Recipes0=R(query)
    for r,r0 in zip(Recipes[:],Recipes0[:]):
        if 'calories' not in r or 'protein' not in r or 'fat' not in r:
            Recipes0.remove(r0)
    Recipes0.sort(key=lambda _:_['cost'])                     # sort low to high               
    Titles=[r['title'] for r in Recipes0]
    return Titles

In [11]:
def search(query,ordering='normal',count=10):
    if ordering=='simple':
        Titles=S(query)
    elif ordering=='healthy':
        Titles=H(query)
    else:
        Titles=N(query)
    L=min(len(Titles),count)
    Titles=Titles[:L]
    for i in range(L):
        print(Titles[i])